## Find Maximum Width and Height in Data


In [2]:
!identify -format "%w %h %f\n" data/memes/*.jpg | sort -n -r -k 1 | head -n 3

8192 4608 6maf6w.jpg
5906 5906 6gkhya.jpg
5312 2988 6myafs.jpg
sort: write failed: 'standard output': Broken pipe
sort: write error


In [3]:
!identify -format "%w %h %f\n" data/memes/*.jpg | sort -n -r -k 2 | head -n 3

700 9333 7px0on.jpg
5906 5906 6gkhya.jpg
2988 5312 79zju3.jpg
sort: write failed: 'standard output': Broken pipe
sort: write error


Max width and height: $(8192, 9333)$


## Load and Preprocess Data

Preprocessing steps:

1. Convert to grayscale
2. Resize to $(224, 224)$


In [66]:
SEED = 160396

In [67]:
import os

import keras

train = keras.utils.image_dataset_from_directory(
    os.path.join("data", "memes"),
    labels=None,
    label_mode=None,
    color_mode="rgb",
    batch_size=None,
    image_size=(224, 224),
    seed=SEED,
    validation_split=0.99,
    subset="training",
)

Found 3326 files.
Using 34 files for training.


## Extract Features

Models used:

1. HOG
2. KAZE
3. PCA
4. Inception v3

All feature sizes are $(2048,)$


### HOG (Histogram of Oriented Gradients)


In [68]:
import tensorflow as tf
from skimage.feature import hog


def get_hog_feature(image):
    return hog(
        image,
        orientations=8,
        pixels_per_cell=(32, 32),
        cells_per_block=(4, 4),
        feature_vector=True,
        channel_axis=-1,
    )


# hog_features = list(map(lambda x: get_hog_feature(x), train.as_numpy_iterator()))
hog_features = train.map(lambda x: tf.py_function(get_hog_feature, [x], tf.float32))

In [69]:
for element in hog_features.take(1):
    print(element.shape)

(2048,)


### KAZE


In [70]:
import numpy as np
import cv2


def get_kaze_feature(image, N=32):
    image = image.numpy()
    alg = cv2.KAZE_create()  # other possibilities are SIFT, ORB

    # Finding image keypoints
    # Number of keypoints varies depending on image size and color palette
    keypts = alg.detect(image)

    # Sorting them based on keypoint response value (bigger is better)
    # Getting first N keypoints
    keypts = sorted(keypts, key=lambda x: -x.response)[:N]

    # Computing descriptors vector
    _, descriptor = alg.compute(image, keypts)
    if descriptor is None:
        descriptor = np.zeros((N * 64,))

    # Flatten all of them in one big vector - our feature vector
    descriptor = descriptor.flatten()

    # Descriptor vector size is 64
    needed_size = N * 64
    if descriptor.size < needed_size:
        # if we have less than N descriptors, then just add zeros at the end
        # of our feature vector
        descriptor = np.concatenate(
            [descriptor, np.zeros(needed_size - descriptor.size)]
        )

    return descriptor


kaze_features = train.map(lambda x: tf.py_function(get_kaze_feature, [x], tf.float32))

In [71]:
for element in kaze_features.take(1):
    print(element.shape)

(2048,)


### PCA


In [74]:
from sklearn.decomposition import PCA

# n_components=0.80 means it will return the Eigenvectors that
# have the 80% of the variation in the dataset
pca = PCA(n_components=0.8)
train_np = np.array(list(train.as_numpy_iterator())).reshape(-1, 224 * 224 * 3)
print(train_np.shape)
pca.fit(train_np)

(34, 150528)


PCA(n_components=0.8)

In [75]:
pca_features = pca.transform(train_np)
print(pca_features.shape)

(34, 13)


### Inception v3

In [80]:
from keras.applications.inception_v3 import InceptionV3, preprocess_input

model = InceptionV3(weights="imagenet", include_top=False, pooling="avg", input_shape=(224, 224, 3))
inception_train = np.array(list((train.map(preprocess_input).as_numpy_iterator())))
inception_features = model.predict(inception_train)
print(inception_features.shape)

2/2 ━━━━━━━━━━━━━━━━━━━━ 17s 7s/step
(34, 2048)


## Cluster with K-Means


In [81]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=3, random_state=SEED)
features = [
    np.array(list(hog_features.as_numpy_iterator())),
    np.array(list(kaze_features.as_numpy_iterator())),
    pca_features,
    inception_features
]

In [82]:
for feature in features:
    preds = kmeans.fit_predict(feature)
    print(preds)

[0 2 1 1 2 2 0 1 2 1 0 1 1 2 1 2 0 0 0 2 0 2 0 0 2 0 0 2 2 2 2 2 2 2]
[1 2 2 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 2 2 2 1 1 1 1 1 1]
[0 0 2 1 0 1 2 1 2 2 1 2 1 1 1 0 1 1 1 1 0 2 2 1 0 2 1 2 2 0 1 2 1 0]
[1 1 2 2 2 2 2 2 1 2 2 2 1 2 2 1 1 2 2 2 0 2 2 2 1 2 2 2 2 1 1 2 1 2]
